# 010. Vectorization of Statement (문장의 vector 화)

- BOW (Bag of Words)
- TF-IDF (Term Frequency - Inverse Document Frequency)  
- Word Embedding - Keras word API 사용

In [1]:
import pandas as pd

sentences = ['I love my dog.',
             'I love my cat.',
             'I love my dog and love my cat',
             'You love my dog!',
             'Do you think my dog is amazing?']

## 1. Bag of Word (BOW)

CountVectorizer는 Python의 scikit-learn 라이브러리에 포함된 클래스로, 텍스트 데이터의 토큰화(tokenization)와 단어 빈도 수를 기반으로 하는 피처 벡터(feature vector)를 생성하는 데 사용됩니다. 이 클래스는 자연어 처리(Natural Language Processing, NLP)와 텍스트 마이닝에서 널리 사용되며, 주요 기능은 다음과 같습니다:

토큰화(Tokenization): 문장이나 문서를 개별 단어나 표현으로 분할합니다.

단어 빈도 계산(Word Frequency Counting): 각 단어가 문서 내에서 나타나는 빈도를 계산합니다.

피처 벡터 생성(Feature Vector Creation): 각 문서를 단어의 빈도를 나타내는 벡터로 변환합니다. 이 벡터는 머신러닝 알고리즘에 입력으로 사용될 수 있습니다.

사전 구축(Vocabulary Building): 모든 문서에서 사용된 모든 단어의 사전을 만듭니다.

CountVectorizer를 사용하면 텍스트 데이터를 수치적으로 분석할 수 있으며, 이는 감정 분석, 주제 모델링, 문서 분류와 같은 다양한 NLP 응용 프로그램에서 중요한 단계입니다. 예를 들어, 스팸 메일 분류, 문서 군집화, 텍스트 기반 추천 시스템 등에 사용됩니다.

- CountVectorizer 주요 파라미터  
    - min_df : vocabulary 에 포함할 최소 발생 빈도
    - ngram_range : (1, 1) - unigram only, (1, 2) - unigram + bigram
    - max_features : top max_features 만으로 vocabulary 구성
    - token_pattern = (?u)\\b\\w\\w+\\b : unocode 영수자 2 글자 이상만 포함

## Text vs token Matrix 생성

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()

# sentences 데이터에 대한 피처 변환 수행
features = count_vectorizer.fit_transform(sentences)
features

<5x10 sparse matrix of type '<class 'numpy.int64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [3]:
print(f"document 수: {features.shape[0]}")
print(f"단어수: {features.shape[1]-1}")

document 수: 5
단어수: 9


In [4]:
# features 객체를 NumPy 배열로 변환
vectorized_sentences = features.toarray()
vectorized_sentences

array([[0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 2, 2, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 1, 1, 1]])

### features 의 단어 list

In [5]:
# CountVectorizer를 통해 추출한 피처(단어) 이름들을 가져옴
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['amazing', 'and', 'cat', 'do', 'dog', 'is', 'love', 'my', 'think',
       'you'], dtype=object)

In [6]:
# 벡터화된 문장과 피처 이름을 이용해 DataFrame 생성
df = pd.DataFrame(vectorized_sentences, columns=feature_names)

# 데이터프레임의 인덱스 이름 지정
df.index.name = 'sentence'
df

,amazing,and,cat,do,dog,is,love,my,think,you
sentence,,,,,,,,,,
0,0,0,0,0,1,0,1,1,0,0
1,0,0,1,0,0,0,1,1,0,0
2,0,1,1,0,1,0,2,2,0,0
3,0,0,0,0,1,0,1,1,0,1
4,1,0,0,1,1,1,0,1,1,1


## 2. TF-IDF

- TF-IDF(Term Frequency - Inverse Document Frequency)  

TF-IDF는 단어의 빈도와 그 단어가 드물게 나타나는 문서에 더 높은 가중치를 부여하는 방식으로 작동합니다.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 객체 생성
tfidf_vectorizer = TfidfVectorizer()

# sentences 데이터에 대한 TF-IDF 기반 피처 변환 수행
tfidf_sentences = tfidf_vectorizer.fit_transform(sentences)
tfidf_sentences

<5x10 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

## Text vs tf-idf Matrix 생성

In [8]:
# TF-IDF 피처 객체를 NumPy 배열로 변환
tfidf_vect_sentences = tfidf_sentences.toarray()
tfidf_vect_sentences

array([[0.        , 0.        , 0.        , 0.        , 0.60685614,
        0.        , 0.60685614, 0.51327503, 0.        , 0.        ],
       [0.        , 0.        , 0.73792244, 0.        , 0.        ,
        0.        , 0.51528988, 0.43582888, 0.        , 0.        ],
       [0.        , 0.49110884, 0.39622352, 0.        , 0.27668216,
        0.        , 0.55336431, 0.46803199, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.45805379,
        0.        , 0.45805379, 0.38741896, 0.        , 0.65595732],
       [0.43872423, 0.        , 0.        , 0.43872423, 0.24716958,
        0.43872423, 0.        , 0.20905445, 0.43872423, 0.35395995]])

In [9]:
# TfidfVectorizer를 통해 추출한 피처(단어) 이름들을 가져옴
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_feature_names

array(['amazing', 'and', 'cat', 'do', 'dog', 'is', 'love', 'my', 'think',
       'you'], dtype=object)

In [10]:
# TF-IDF 벡터화된 문장과 피처 이름을 이용해 DataFrame 생성
df = pd.DataFrame(tfidf_vect_sentences, columns=tfidf_feature_names)
df

,amazing,and,cat,do,dog,is,love,my,think,you
0,0.000000,0.000000,0.000000,0.000000,0.606856,0.000000,0.606856,0.513275,0.000000,0.000000
1,0.000000,0.000000,0.737922,0.000000,0.000000,0.000000,0.515290,0.435829,0.000000,0.000000
2,0.000000,0.491109,0.396224,0.000000,0.276682,0.000000,0.553364,0.468032,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.458054,0.000000,0.458054,0.387419,0.000000,0.655957
4,0.438724,0.000000,0.000000,0.438724,0.247170,0.438724,0.000000,0.209054,0.438724,0.353960


# 3. keras word encoding

- keras  API 이용

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

## Tokenize

In [12]:
# 문장으로 부터 상위 100 개 단어로 vocabulary 작성
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')

## Word Index Vocabulary 작성

In [13]:
# sentences에 포함된 문장들을 기반으로 단어의 토큰화를
# 수행하며, 각 단어에 고유한 인덱스를 할당
tokenizer.fit_on_texts(sentences)

# 각 단어에 부여된 고유 인덱스 추출
print(tokenizer.word_index)
print(tokenizer.index_word)

{'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'cat': 6, 'you': 7, 'and': 8, 'do': 9, 'think': 10, 'is': 11, 'amazing': 12}
{1: '<OOV>', 2: 'my', 3: 'love', 4: 'dog', 5: 'i', 6: 'cat', 7: 'you', 8: 'and', 9: 'do', 10: 'think', 11: 'is', 12: 'amazing'}


## text 의 sentence 변환 및 paddding

- texts_to_sequences: text list 내의 각 text 를 수열 (sequence of integers) 로 convert


    - 입력 : text (strings) list
    - 반환 : sequence list
    
- pad_sequences: 동일한 길이로 sequence 를 zero padding

In [14]:
# sentences 데이터를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(sentences)

# 시퀀스에 패딩 적용 (문장의 뒤쪽을 패딩하고, 필요시 뒤쪽을 잘라냄)
padded = pad_sequences(sequences, padding='post', truncating='post')

print(sequences)
print()
print(padded)

[[5, 3, 2, 4], [5, 3, 2, 6], [5, 3, 2, 4, 8, 3, 2, 6], [7, 3, 2, 4], [9, 7, 10, 2, 4, 11, 12]]

[[ 5  3  2  4  0  0  0  0]
 [ 5  3  2  6  0  0  0  0]
 [ 5  3  2  4  8  3  2  6]
 [ 7  3  2  4  0  0  0  0]
 [ 9  7 10  2  4 11 12  0]]


### sequenced sentence 를 word sentence 로 환원

In [15]:
for sequence in sequences:
    sent = []
    for idx in sequence:
        sent.append(tokenizer.index_word[idx])
    print(' '.join(sent))

i love my dog
i love my cat
i love my dog and love my cat
you love my dog
do you think my dog is amazing


### One-Hot-Encoding 표현

In [16]:
to_categorical(padded)

array([[[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

    